In [1]:
import pandas as pd
import numpy as np

In [2]:
nike_df = pd.read_csv("dataset/nike_2020_04_13.csv")
adidas_df = pd.read_csv("dataset/Adidas final.csv", delimiter=";")

In [3]:
adidas_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2625 entries, 0 to 2624
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   URL            2625 non-null   object
 1   Product Name   2625 non-null   object
 2   Product ID     2625 non-null   object
 3   Listing Price  2625 non-null   int64 
 4   Sale Price     2625 non-null   int64 
 5   Discount       2625 non-null   int64 
 6   Brand          2625 non-null   object
 7   Description    2625 non-null   object
 8   Rating         2625 non-null   int64 
 9   Reviews        2625 non-null   int64 
 10  Images         2625 non-null   object
 11  Last Visited   2625 non-null   object
dtypes: int64(5), object(7)
memory usage: 246.2+ KB


In [4]:
nike_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 643 entries, 0 to 642
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   URL            643 non-null    object 
 1   Product Name   643 non-null    object 
 2   Product ID     643 non-null    object 
 3   Listing Price  643 non-null    int64  
 4   Sale Price     643 non-null    int64  
 5   Discount       643 non-null    int64  
 6   Brand          643 non-null    object 
 7   Description    640 non-null    object 
 8   Rating         643 non-null    float64
 9   Reviews        643 non-null    int64  
 10  Images         572 non-null    object 
 11  Last Visited   643 non-null    object 
dtypes: float64(1), int64(4), object(7)
memory usage: 60.4+ KB


In [6]:
useful_cols = ["Product Name", "Product ID", "Sale Price", "Brand", "Description"]
my_nike  = nike_df[useful_cols]

In [7]:
my_nike["Sale Price"].isnull().values.any()

False

In [13]:
for col in useful_cols:
    print(col + " " + str(my_nike[col].isnull().sum()))

Product Name 0
Product ID 0
Sale Price 0
Brand 0
Description 3


In [9]:
my_nike[my_nike.isna().any(axis=1)]

,Product Name,Product ID,Sale Price,Brand,Description
155,Nike React Infinity Run Flyknit,CD4371-001,13995,Nike,NaN
263,Nike React Infinity Run Flyknit,CD4372-004,13995,Nike,NaN
508,Nike Free X Metcon 2,CD8526-446,9995,Nike,NaN


In [15]:
for feature in useful_cols:
    my_nike[feature] = my_nike[feature].fillna('')

<ipython-input-15-5727e342ca68>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  my_nike[feature] = my_nike[feature].fillna('')


In [17]:
for col in useful_cols:
    print(col + " " + str(my_nike[col].isnull().sum()))

Product Name 0
Product ID 0
Sale Price 0
Brand 0
Description 0


In [20]:
my_nike.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 643 entries, 0 to 642
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Product Name  643 non-null    object
 1   Product ID    643 non-null    object
 2   Sale Price    643 non-null    int64 
 3   Brand         643 non-null    object
 4   Description   643 non-null    object
dtypes: int64(1), object(4)
memory usage: 25.2+ KB


In [21]:
# "Product Name", "Product ID", "Sale Price", "Brand", "Description"
def combined_features(row):
    return row['Product Name']+" "+str(row['Sale Price'])+" "+row['Brand']+" "+row['Description']

In [22]:
my_nike["combined_features"] = my_nike.apply(combined_features, axis =1)

<ipython-input-22-f73ff0d15cf4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  my_nike["combined_features"] = my_nike.apply(combined_features, axis =1)


In [24]:
my_nike.iloc[0]['combined_features']

"Nike Air Force 1 '07 Essential 7495 Nike Let your shoe game shimmer in the Nike Air Force 1 '07 Essential. It takes the classic AF-1 design to the next level with its premium leather upper and iridescent Swoosh."

In [25]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [1]:
cv = CountVectorizer(stop_word='english')
count_matrix = cv.fit_transform(my_nike["combined_features"])
print("Count Matrix:", count_matrix.toarray())

NameError: name 'CountVectorizer' is not defined

In [27]:
cosine_sim = cosine_similarity(count_matrix)

In [29]:
import seaborn as sns
import matplotlib.pylab as plt

array([[1.        , 0.6491507 , 0.41718848, ..., 0.37851665, 0.52654816,
        0.32748932],
       [0.6491507 , 1.        , 0.40576303, ..., 0.38110797, 0.54371495,
        0.32698355],
       [0.41718848, 0.40576303, 1.        , ..., 0.24002743, 0.38437111,
        0.32881028],
       ...,
       [0.37851665, 0.38110797, 0.24002743, ..., 1.        , 0.46549167,
        0.33646329],
       [0.52654816, 0.54371495, 0.38437111, ..., 0.46549167, 1.        ,
        0.39366865],
       [0.32748932, 0.32698355, 0.32881028, ..., 0.33646329, 0.39366865,
        1.        ]])

In [38]:
similar_movies = list(enumerate(cosine_sim[1]))

In [39]:
sorted_similar_movies = sorted(similar_movies, key=lambda x:x[1], reverse=True)

In [41]:
sorted_similar_movies[1]

(172, 0.8911327886790066)

In [42]:
my_nike.iloc[1]

Product Name                                      Nike Air Force 1 '07
Product ID                                                  CT4328-101
Sale Price                                                        7495
Brand                                                             Nike
Description          The legend lives on in the Nike Air Force 1 '0...
combined_features    Nike Air Force 1 '07 7495 Nike The legend live...
Name: 1, dtype: object

In [43]:
my_nike.iloc[172]

Product Name                                Nike Air Force 1 Low Retro
Product ID                                                  CQ0492-001
Sale Price                                                       10995
Brand                                                             Nike
Description          The legend lives on in the Nike Air Force 1 '0...
combined_features    Nike Air Force 1 Low Retro 10995 Nike The lege...
Name: 172, dtype: object